In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox
from cryptography.fernet import Fernet
from PIL import Image, ImageTk
import io
import os

class FileEncryptionApp:
    def __init__(self, root):
        # Initialize the application
        self.root = root
        self.root.title("Application for Encryption and Decryption")

        # Generate a random key for encryption/decryption
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

        # Create GUI elements - Buttons for file operations and folder operations
        self.encrypt_button = tk.Button(root, text="Encrypt a Text File", command=self.encrypt_file)
        self.encrypt_button.pack()

        self.decrypt_button = tk.Button(root, text="Decrypt a Text File", command=self.decrypt_file)
        self.decrypt_button.pack()

        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder)
        self.encrypt_folder_button.pack()

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder)
        self.decrypt_folder_button.pack()

        # Create GUI elements - Buttons for image operations
        self.encrypt_image_button = tk.Button(root, text="Encrypt an JPG Image", command=self.encrypt_image)
        self.encrypt_image_button.pack()

        self.decrypt_image_button = tk.Button(root, text="Decrypt an JPG Image", command=self.decrypt_image)
        self.decrypt_image_button.pack()

        # Create a label for displaying encrypted/decrypted images
        self.image_label = tk.Label(root, text="Encrypted/Decrypted Image:")
        self.image_label.pack()

    def encrypt_file(self):
        # Ask user to select a file for encryption
        file_path = filedialog.askopenfilename(title="Select File to Encrypt", filetypes=[("All Files", "*.*")])
        if file_path:
            # Read the content of the selected file
            with open(file_path, 'rb') as file:
                data = file.read()
                # Encrypt the file content
                encrypted_data = self.cipher.encrypt(data)

            # Ask user to select a location to save the encrypted file
            encrypted_file_path = filedialog.asksaveasfilename(title="Save Encrypted File", defaultextension="enc",
                                                                filetypes=[("Encrypted Files", "*.enc")])
            if encrypted_file_path:
                # Save the encrypted content to the specified location
                with open(encrypted_file_path, 'wb') as encrypted_file:
                    encrypted_file.write(encrypted_data)
                    messagebox.showinfo("Success", "File encrypted successfully.")

    def decrypt_file(self):
        # Ask user to select an encrypted file for decryption
        file_path = filedialog.askopenfilename(title="Select File to Decrypt", filetypes=[("Encrypted Files", "*.enc")])
        if file_path:
            # Read the content of the encrypted file
            with open(file_path, 'rb') as encrypted_file:
                encrypted_data = encrypted_file.read()
                # Decrypt the file content
                decrypted_data = self.cipher.decrypt(encrypted_data)

            # Ask user to select a location to save the decrypted file
            decrypted_file_path = filedialog.asksaveasfilename(title="Save Decrypted File",
                                                                filetypes=[("All Files", "*.*")])
            if decrypted_file_path:
                # Save the decrypted content to the specified location
                with open(decrypted_file_path, 'wb') as decrypted_file:
                    decrypted_file.write(decrypted_data)
                    messagebox.showinfo("Success", "File decrypted successfully.")

    def encrypt_image(self):
        # Ask user to select an image file for encryption
        image_path = filedialog.askopenfilename(title="Select Image to Encrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            # Read the content of the selected image file
            with open(image_path, 'rb') as image_file:
                image_data = image_file.read()
                # Encrypt the image content
                encrypted_data = self.cipher.encrypt(image_data)

            # Display the encrypted image in the GUI
            encrypted_image = Image.open(io.BytesIO(encrypted_data))
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            messagebox.showinfo("Success", "Image encrypted successfully.")

    def decrypt_image(self):
        # Ask user to select an image file for decryption
        image_path = filedialog.askopenfilename(title="Select Image to Decrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            # Read the content of the encrypted image file
            with open(image_path, 'rb') as encrypted_image_file:
                encrypted_data = encrypted_image_file.read()
                # Decrypt the image content
                decrypted_data = self.cipher.decrypt(encrypted_data)

            # Display the decrypted image in the GUI
            decrypted_image = Image.open(io.BytesIO(decrypted_data))
            decrypted_image.thumbnail((300, 300))
            decrypted_image_tk = ImageTk.PhotoImage(decrypted_image)
            self.image_label.config(image=decrypted_image_tk)
            self.image_label.image = decrypted_image_tk

            messagebox.showinfo("Success", "Image decrypted successfully.")

    def encrypt_folder(self):
        # Ask user to select a folder for encryption
        folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
        if folder_path:
            # Traverse through all files in the folder and encrypt each file
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.encrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        # Ask user to select a folder for decryption
        folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
        if folder_path:
            # Traverse through all files in the folder and decrypt each file
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.decrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder decrypted successfully.")

    def encrypt_file_path(self, file_path):
        # Encrypt individual files within a folder
        with open(file_path, 'rb') as file:
            data = file.read()
            encrypted_data = self.cipher.encrypt(data)

        # Save the encrypted content with a '.enc' extension
        encrypted_file_path = file_path + ".enc"
        with open(encrypted_file_path, 'wb') as encrypted_file:
            encrypted_file.write(encrypted_data)

    def decrypt_file_path(self, file_path):
        # Decrypt individual files within a folder
        with open(file_path, 'rb') as encrypted_file:
            encrypted_data = encrypted_file.read()
            decrypted_data = self.cipher.decrypt(encrypted_data)

        # Save the decrypted content without the '.enc' extension
        decrypted_file_path = file_path[:-4]  # Remove the '.enc' extension
        with open(decrypted_file_path, 'wb') as decrypted_file:
            decrypted_file.write(decrypted_data)

if __name__ == "__main__":
    # Create the Tkinter root window and run the application
    root = tk.Tk()
    app = FileEncryptionApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/2419797133.py", line 91, in encrypt_image
    encrypted_image = Image.open(io.BytesIO(encrypted_data))
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 3030, in open
    raise UnidentifiedImageError(
PIL.UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002AEABE35130>


solving image error

In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox
from cryptography.fernet import Fernet
from PIL import Image, ImageTk
import io
import os

class FileEncryptionApp:
    def __init__(self, root):
        # Initialize the application
        self.root = root
        self.root.title("File Encryption App")

        # Generate a random key for encryption/decryption
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

        # Create GUI elements - Buttons for file operations and folder operations
        self.encrypt_button = tk.Button(root, text="Encrypt File", command=self.encrypt_file)
        self.encrypt_button.pack()

        self.decrypt_button = tk.Button(root, text="Decrypt File", command=self.decrypt_file)
        self.decrypt_button.pack()

        self.encrypt_folder_button = tk.Button(root, text="Encrypt Folder", command=self.encrypt_folder)
        self.encrypt_folder_button.pack()

        self.decrypt_folder_button = tk.Button(root, text="Decrypt Folder", command=self.decrypt_folder)
        self.decrypt_folder_button.pack()

        # Create GUI elements - Buttons for image operations
        self.encrypt_image_button = tk.Button(root, text="Encrypt Image", command=self.encrypt_image)
        self.encrypt_image_button.pack()

        self.decrypt_image_button = tk.Button(root, text="Decrypt Image", command=self.decrypt_image)
        self.decrypt_image_button.pack()

        # Create a label for displaying encrypted/decrypted images
        self.image_label = tk.Label(root, text="Encrypted/Decrypted Image:")
        self.image_label.pack()

    def encrypt_file(self):
        # Ask user to select a file for encryption
        file_path = filedialog.askopenfilename(title="Select File to Encrypt", filetypes=[("All Files", "*.*")])
        if file_path:
            # Read the content of the selected file
            with open(file_path, 'rb') as file:
                data = file.read()
                # Encrypt the file content
                encrypted_data = self.cipher.encrypt(data)

            # Ask user to select a location to save the encrypted file
            encrypted_file_path = filedialog.asksaveasfilename(title="Save Encrypted File", defaultextension="enc",
                                                                filetypes=[("Encrypted Files", "*.enc")])
            if encrypted_file_path:
                # Save the encrypted content to the specified location
                with open(encrypted_file_path, 'wb') as encrypted_file:
                    encrypted_file.write(encrypted_data)
                    messagebox.showinfo("Success", "File encrypted successfully.")

    def decrypt_file(self):
        # Ask user to select an encrypted file for decryption
        file_path = filedialog.askopenfilename(title="Select File to Decrypt", filetypes=[("Encrypted Files", "*.enc")])
        if file_path:
            # Read the content of the encrypted file
            with open(file_path, 'rb') as encrypted_file:
                encrypted_data = encrypted_file.read()
                # Decrypt the file content
                decrypted_data = self.cipher.decrypt(encrypted_data)

            # Ask user to select a location to save the decrypted file
            decrypted_file_path = filedialog.asksaveasfilename(title="Save Decrypted File",
                                                                filetypes=[("All Files", "*.*")])
            if decrypted_file_path:
                # Save the decrypted content to the specified location
                with open(decrypted_file_path, 'wb') as decrypted_file:
                    decrypted_file.write(decrypted_data)
                    messagebox.showinfo("Success", "File decrypted successfully.")

    def encrypt_image(self):
        # Ask user to select an image file for encryption
        image_path = filedialog.askopenfilename(title="Select Image to Encrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            # Display the original image before encryption
            original_image = Image.open(image_path)
            original_image.thumbnail((300, 300))
            original_image_tk = ImageTk.PhotoImage(original_image)
            self.image_label.config(image=original_image_tk)
            self.image_label.image = original_image_tk

            # Read the content of the selected image file
            with open(image_path, 'rb') as image_file:
                image_data = image_file.read()
                # Encrypt the image content
                encrypted_data = self.cipher.encrypt(image_data)

            # Display the encrypted image in the GUI
            encrypted_image = Image.open(io.BytesIO(encrypted_data))
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            messagebox.showinfo("Success", "Image encrypted successfully.")

    def decrypt_image(self):
        # Ask user to select an image file for decryption
        image_path = filedialog.askopenfilename(title="Select Image to Decrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            # Display the encrypted image before decryption
            encrypted_image = Image.open(image_path)
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            # Read the content of the encrypted image file
            with open(image_path, 'rb') as encrypted_image_file:
                encrypted_data = encrypted_image_file.read()
                # Decrypt the image content
                decrypted_data = self.cipher.decrypt(encrypted_data)

            # Display the decrypted image in the GUI
            decrypted_image = Image.open(io.BytesIO(decrypted_data))
            decrypted_image.thumbnail((300, 300))
            decrypted_image_tk = ImageTk.PhotoImage(decrypted_image)
            self.image_label.config(image=decrypted_image_tk)
            self.image_label.image = decrypted_image_tk

            messagebox.showinfo("Success", "Image decrypted successfully.")

    def encrypt_folder(self):
        # Ask user to select a folder for encryption
        folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
        if folder_path:
            # Traverse through all files in the folder and encrypt each file
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.encrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        # Ask user to select a folder for decryption
        folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
        if folder_path:
            # Traverse through all files in the folder and decrypt each file
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.decrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder decrypted successfully.")

    def encrypt_file_path(self, file_path):
        # Encrypt individual files within a folder
        with open(file_path, 'rb') as file:
            data = file.read()
            encrypted_data = self.cipher.encrypt(data)

        # Save the encrypted content with a '.enc' extension
        encrypted_file_path = file_path + ".enc"
        with open(encrypted_file_path, 'wb') as encrypted_file:
            encrypted_file.write(encrypted_data)

    def decrypt_file_path(self, file_path):
        # Decrypt individual files within a folder
        with open(file_path, 'rb') as encrypted_file:
            encrypted_data = encrypted_file.read()
            decrypted_data = self.cipher.decrypt(encrypted_data)

        # Save the decrypted content without the '.enc' extension
        decrypted_file_path = file_path[:-4]  # Remove the '.enc' extension
        with open(decrypted_file_path, 'wb') as decrypted_file:
            decrypted_file.write(decrypted_data)

if __name__ == "__main__":
    # Create the Tkinter root window and run the application
    root = tk.Tk()
    app = FileEncryptionApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/1631872397.py", line 98, in encrypt_image
    encrypted_image = Image.open(io.BytesIO(encrypted_data))
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 3030, in open
    raise UnidentifiedImageError(
PIL.UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002AEAA6D7F90>
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/1631872397.py", line 98, in encrypt_image
    encrypted_image = Image.open(io.BytesIO(encrypted_data))
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 3030, in open
    raise Unidentifi

making fancy gui

In [9]:
import tkinter as tk
from tkinter import filedialog, messagebox
from cryptography.fernet import Fernet
from PIL import Image, ImageTk
import io
import os

class FileEncryptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Application for Encryption and Decryption")

        # Set a custom background color
        self.root.configure(bg="#e0e0e0")

        # Generate a random key for encryption/decryption
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#4CAF50", 'fg': "white", 'font': ('Helvetica', 12)}

        # Create buttons for file operations
        self.encrypt_button = tk.Button(root, text="Encrypt a Text File", command=self.encrypt_file, **button_style)
        self.encrypt_button.pack(pady=10)

        self.decrypt_button = tk.Button(root, text="Decrypt a Text File", command=self.decrypt_file, **button_style)
        self.decrypt_button.pack(pady=10)

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.pack(pady=10)

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.pack(pady=10)

        # Create buttons for image operations
        self.encrypt_image_button = tk.Button(root, text="Encrypt a JPG Image", command=self.encrypt_image, **button_style)
        self.encrypt_image_button.pack(pady=10)

        self.decrypt_image_button = tk.Button(root, text="Decrypt a JPG Image", command=self.decrypt_image, **button_style)
        self.decrypt_image_button.pack(pady=10)

        # Create a label for displaying encrypted/decrypted images
        self.image_label = tk.Label(root, text="Encrypted/Decrypted Image:", font=('Helvetica', 14), bg="#e0e0e0")
        self.image_label.pack()

    def encrypt_file(self):
        file_path = filedialog.askopenfilename(title="Select File to Encrypt", filetypes=[("All Files", "*.*")])
        if file_path:
            with open(file_path, 'rb') as file:
                data = file.read()
                encrypted_data = self.cipher.encrypt(data)

            encrypted_file_path = filedialog.asksaveasfilename(title="Save Encrypted File", defaultextension="enc",
                                                                filetypes=[("Encrypted Files", "*.enc")])
            if encrypted_file_path:
                with open(encrypted_file_path, 'wb') as encrypted_file:
                    encrypted_file.write(encrypted_data)
                    messagebox.showinfo("Success", "File encrypted successfully.")

    def decrypt_file(self):
        file_path = filedialog.askopenfilename(title="Select File to Decrypt", filetypes=[("Encrypted Files", "*.enc")])
        if file_path:
            with open(file_path, 'rb') as encrypted_file:
                encrypted_data = encrypted_file.read()
                decrypted_data = self.cipher.decrypt(encrypted_data)

            decrypted_file_path = filedialog.asksaveasfilename(title="Save Decrypted File",
                                                                filetypes=[("All Files", "*.*")])
            if decrypted_file_path:
                with open(decrypted_file_path, 'wb') as decrypted_file:
                    decrypted_file.write(decrypted_data)
                    messagebox.showinfo("Success", "File decrypted successfully.")

    def encrypt_image(self):
        image_path = filedialog.askopenfilename(title="Select Image to Encrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            original_image = Image.open(image_path)
            original_image.thumbnail((300, 300))
            original_image_tk = ImageTk.PhotoImage(original_image)
            self.image_label.config(image=original_image_tk)
            self.image_label.image = original_image_tk

            with open(image_path, 'rb') as image_file:
                image_data = image_file.read()
                encrypted_data = self.cipher.encrypt(image_data)

            encrypted_image = Image.open(io.BytesIO(encrypted_data))
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            messagebox.showinfo("Success", "Image encrypted successfully.")

    def decrypt_image(self):
        image_path = filedialog.askopenfilename(title="Select Image to Decrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            encrypted_image = Image.open(image_path)
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            with open(image_path, 'rb') as encrypted_image_file:
                encrypted_data = encrypted_image_file.read()
                decrypted_data = self.cipher.decrypt(encrypted_data)

            decrypted_image = Image.open(io.BytesIO(decrypted_data))
            decrypted_image.thumbnail((300, 300))
            decrypted_image_tk = ImageTk.PhotoImage(decrypted_image)
            self.image_label.config(image=decrypted_image_tk)
            self.image_label.image = decrypted_image_tk

            messagebox.showinfo("Success", "Image decrypted successfully.")

    def encrypt_folder(self):
        folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
        if folder_path:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.encrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
        if folder_path:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.decrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder decrypted successfully.")

    def encrypt_file_path(self, file_path):
        with open(file_path, 'rb') as file:
            data = file.read()
            encrypted_data = self.cipher.encrypt(data)

        encrypted_file_path = file_path + ".enc"
        with open(encrypted_file_path, 'wb') as encrypted_file:
            encrypted_file.write(encrypted_data)

    def decrypt_file_path(self, file_path):
        with open(file_path, 'rb') as encrypted_file:
            encrypted_data = encrypted_file.read()
            decrypted_data = self.cipher.decrypt(encrypted_data)

        decrypted_file_path = file_path[:-4]
        with open(decrypted_file_path, 'wb') as decrypted_file:
            decrypted_file.write(decrypted_data)

if __name__ == "__main__":
    root = tk.Tk()
    app = FileEncryptionApp(root)
    root.mainloop()


In [13]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from PIL import Image, ImageTk
import os

class FolderEncryptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Folder Encryption and Decryption")

        # Set a custom background color
        self.root.configure(bg="#e0e0e0")

        # Load Qatar logo as background
        self.qatar_logo = Image.open("./qatar1.jpg")  # Replace with the path to your logo image
        self.qatar_logo = ImageTk.PhotoImage(self.qatar_logo)

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#4CAF50", 'fg': "white", 'font': ('Helvetica', 12)}

        # Set background image
        bg_label = tk.Label(root, image=self.qatar_logo)
        bg_label.place(relwidth=1, relheight=1)

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.place(relx=0.1, rely=0.7)

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.place(relx=0.4, rely=0.7)

    def get_password(self):
        # Prompt user for password
        password = simpledialog.askstring("Password", "Enter password:", show='*')
        return password.encode('utf-8')

    def derive_key(self, password):
        # Derive a key from the password using PBKDF2HMAC
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'\xae\xd6c}\x93\xa6\xcc\xc2\n\xd4\x98\x16\xc6y\x8f',  # Use a different salt for each application
            iterations=100000,
            backend=default_backend()
        )
        key = Fernet(kdf.derive(password))
        return key

    def encrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as file:
                            data = file.read()
                            encrypted_data = cipher.encrypt(data)

                        encrypted_file_path = file_path + ".enc"
                        with open(encrypted_file_path, 'wb') as encrypted_file:
                            encrypted_file.write(encrypted_data)

                messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as encrypted_file:
                            encrypted_data = encrypted_file.read()
                            decrypted_data = cipher.decrypt(encrypted_data)

                        decrypted_file_path = file_path[:-4]
                        with open(decrypted_file_path, 'wb') as decrypted_file:
                            decrypted_file.write(decrypted_data)

                messagebox.showinfo("Success", "Folder decrypted successfully.")

if __name__ == "__main__":
    root = tk.Tk()
    app = FolderEncryptionApp(root)
    root.mainloop()


TclError: image "pyimage4" doesn't exist

In [15]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os

class FolderEncryptionApp:
    def __init__(self, root):
        # Initialize the main window
        self.root = root
        self.root.title("Folder Encryption and Decryption")

        # Set a custom background color (Maroon)
        self.root.configure(bg="#800000")

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#4CAF50", 'fg': "white", 'font': ('Helvetica', 12)}

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.place(relx=0.1, rely=0.5)  # Place button at specified position

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.place(relx=0.4, rely=0.5)  # Place button at specified position

    def get_password(self):
        # Prompt user for password
        password = simpledialog.askstring("Password", "Enter password:", show='*')
        return password.encode('utf-8')

    def derive_key(self, password):
        # Derive a key from the password using PBKDF2HMAC
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'\xae\xd6c}\x93\xa6\xcc\xc2\n\xd4\x98\x16\xc6y\x8f',  # Use a different salt for each application
            iterations=100000,
            backend=default_backend()
        )
        key = Fernet(kdf.derive(password))
        return key

    def encrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and encrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as file:
                            data = file.read()
                            encrypted_data = cipher.encrypt(data)

                        encrypted_file_path = file_path + ".enc"
                        with open(encrypted_file_path, 'wb') as encrypted_file:
                            encrypted_file.write(encrypted_data)

                messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and decrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as encrypted_file:
                            encrypted_data = encrypted_file.read()
                            decrypted_data = cipher.decrypt(encrypted_data)

                        decrypted_file_path = file_path[:-4]
                        with open(decrypted_file_path, 'wb') as decrypted_file:
                            decrypted_file.write(decrypted_data)

                messagebox.showinfo("Success", "Folder decrypted successfully.")

if __name__ == "__main__":
    # Create and run the application
    root = tk.Tk()
    app = FolderEncryptionApp(root)
    root.mainloop()


In [16]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os

class FolderEncryptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Folder Encryption and Decryption")

        # Initialize background colors
        self.background_colors = ["#FFFFFF", "#800000"]  # White, Maroon

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#000080", 'fg': "white", 'font': ('Helvetica', 12)}

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.pack(pady=10)  # Use pack for vertical alignment

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.pack(pady=10)  # Use pack for vertical alignment

        # Alternate background colors
        for i in range(2):
            self.root.after(1000 * i, lambda i=i: self.root.configure(bg=self.background_colors[i % 2]))

    def get_password(self):
        # Prompt user for password
        password = simpledialog.askstring("Password", "Enter password:", show='*')
        return password.encode('utf-8')

    def derive_key(self, password):
        # Derive a key from the password using PBKDF2HMAC
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'\xae\xd6c}\x93\xa6\xcc\xc2\n\xd4\x98\x16\xc6y\x8f',  # Use a different salt for each application
            iterations=100000,
            backend=default_backend()
        )
        key = Fernet(kdf.derive(password))
        return key

    def encrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and encrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as file:
                            data = file.read()
                            encrypted_data = cipher.encrypt(data)

                        encrypted_file_path = file_path + ".enc"
                        with open(encrypted_file_path, 'wb') as encrypted_file:
                            encrypted_file.write(encrypted_data)

                messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and decrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as encrypted_file:
                            encrypted_data = encrypted_file.read()
                            decrypted_data = cipher.decrypt(encrypted_data)

                        decrypted_file_path = file_path[:-4]
                        with open(decrypted_file_path, 'wb') as decrypted_file:
                            decrypted_file.write(decrypted_data)

                messagebox.showinfo("Success", "Folder decrypted successfully.")

if __name__ == "__main__":
    root = tk.Tk()
    app = FolderEncryptionApp(root)
    root.mainloop()


In [18]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os

class FolderEncryptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Folder Encryption and Decryption")

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#000080", 'fg': "white", 'font': ('Helvetica', 12)}

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.pack(pady=10)  # Use pack for vertical alignment

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.pack(pady=10)  # Use pack for vertical alignment

        # Split the background between white and maroon
        self.root.configure(bg="#FFFFFF")
        self.canvas = tk.Canvas(root, bg="#800000", height=self.root.winfo_screenheight() / 2, width=self.root.winfo_screenwidth())
        self.canvas.pack()

    def get_password(self):
        # Prompt user for password
        password = simpledialog.askstring("Password", "Enter password:", show='*')
        return password.encode('utf-8')

    def derive_key(self, password):
        # Derive a key from the password using PBKDF2HMAC
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'\xae\xd6c}\x93\xa6\xcc\xc2\n\xd4\x98\x16\xc6y\x8f',  # Use a different salt for each application
            iterations=100000,
            backend=default_backend()
        )
        key = Fernet(kdf.derive(password))
        return key

    def encrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and encrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as file:
                            data = file.read()
                            encrypted_data = cipher.encrypt(data)

                        encrypted_file_path = file_path + ".enc"
                        with open(encrypted_file_path, 'wb') as encrypted_file:
                            encrypted_file.write(encrypted_data)

                messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and decrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as encrypted_file:
                            encrypted_data = encrypted_file.read()
                            decrypted_data = cipher.decrypt(encrypted_data)

                        decrypted_file_path = file_path[:-4]
                        with open(decrypted_file_path, 'wb') as decrypted_file:
                            decrypted_file.write(decrypted_data)

                messagebox.showinfo("Success", "Folder decrypted successfully.")

if __name__ == "__main__":
    root = tk.Tk()
    app = FolderEncryptionApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/1235466968.py", line 51, in encrypt_folder
    key = self.derive_key(password)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/1235466968.py", line 43, in derive_key
    key = Fernet(kdf.derive(password))
  File "C:\ProgramData\Anaconda3\lib\site-packages\cryptography\fernet.py", line 34, in __init__
    raise ValueError(
ValueError: Fernet key must be 32 url-safe base64-encoded bytes.
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/1235466968.py", line 51, in encrypt_folder
    key = self.derive_key(password)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipy

In [19]:
import tkinter as tk
from tkinter import filedialog, messagebox
from cryptography.fernet import Fernet
from PIL import Image, ImageTk
import io
import os

class FileEncryptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Application for Encryption and Decryption")

        # Set a custom background color split half-and-half between white and maroon
        self.root.configure(bg="#FFFFFF")
        self.canvas = tk.Canvas(root, bg="#800000", height=self.root.winfo_screenheight(), width=self.root.winfo_screenwidth() / 2)
        self.canvas.pack(side=tk.LEFT)

        # Generate a random key for encryption/decryption
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#000080", 'fg': "white", 'font': ('Helvetica', 12)}

        # Create buttons for file operations
        self.encrypt_button = tk.Button(root, text="Encrypt a Text File", command=self.encrypt_file, **button_style)
        self.encrypt_button.pack(pady=10)

        self.decrypt_button = tk.Button(root, text="Decrypt a Text File", command=self.decrypt_file, **button_style)
        self.decrypt_button.pack(pady=10)

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.pack(pady=10)

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.pack(pady=10)

        # Create buttons for image operations
        self.encrypt_image_button = tk.Button(root, text="Encrypt a JPG Image", command=self.encrypt_image, **button_style)
        self.encrypt_image_button.pack(pady=10)

        self.decrypt_image_button = tk.Button(root, text="Decrypt a JPG Image", command=self.decrypt_image, **button_style)
        self.decrypt_image_button.pack(pady=10)

        # Create a label for displaying encrypted/decrypted images
        self.image_label = tk.Label(root, text="Encrypted/Decrypted Image:", font=('Helvetica', 14), bg="#FFFFFF")
        self.image_label.pack()

    def encrypt_file(self):
        file_path = filedialog.askopenfilename(title="Select File to Encrypt", filetypes=[("All Files", "*.*")])
        if file_path:
            with open(file_path, 'rb') as file:
                data = file.read()
                encrypted_data = self.cipher.encrypt(data)

            encrypted_file_path = filedialog.asksaveasfilename(title="Save Encrypted File", defaultextension="enc",
                                                                filetypes=[("Encrypted Files", "*.enc")])
            if encrypted_file_path:
                with open(encrypted_file_path, 'wb') as encrypted_file:
                    encrypted_file.write(encrypted_data)
                    messagebox.showinfo("Success", "File encrypted successfully.")

    def decrypt_file(self):
        file_path = filedialog.askopenfilename(title="Select File to Decrypt", filetypes=[("Encrypted Files", "*.enc")])
        if file_path:
            with open(file_path, 'rb') as encrypted_file:
                encrypted_data = encrypted_file.read()
                decrypted_data = self.cipher.decrypt(encrypted_data)

            decrypted_file_path = filedialog.asksaveasfilename(title="Save Decrypted File",
                                                                filetypes=[("All Files", "*.*")])
            if decrypted_file_path:
                with open(decrypted_file_path, 'wb') as decrypted_file:
                    decrypted_file.write(decrypted_data)
                    messagebox.showinfo("Success", "File decrypted successfully.")

    def encrypt_image(self):
        image_path = filedialog.askopenfilename(title="Select Image to Encrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            original_image = Image.open(image_path)
            original_image.thumbnail((300, 300))
            original_image_tk = ImageTk.PhotoImage(original_image)
            self.image_label.config(image=original_image_tk)
            self.image_label.image = original_image_tk

            with open(image_path, 'rb') as image_file:
                image_data = image_file.read()
                encrypted_data = self.cipher.encrypt(image_data)

            encrypted_image = Image.open(io.BytesIO(encrypted_data))
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            messagebox.showinfo("Success", "Image encrypted successfully.")

    def decrypt_image(self):
        image_path = filedialog.askopenfilename(title="Select Image to Decrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            encrypted_image = Image.open(image_path)
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            with open(image_path, 'rb') as encrypted_image_file:
                encrypted_data = encrypted_image_file.read()
                decrypted_data = self.cipher.decrypt(encrypted_data)

            decrypted_image = Image.open(io.BytesIO(decrypted_data))
            decrypted_image.thumbnail((300, 300))
            decrypted_image_tk = ImageTk.PhotoImage(decrypted_image)
            self.image_label.config(image=decrypted_image_tk)
            self.image_label.image = decrypted_image_tk

            messagebox.showinfo("Success", "Image decrypted successfully.")

    def encrypt_folder(self):
        folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
        if folder_path:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.encrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
        if folder_path:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.decrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder decrypted successfully.")

    def encrypt_file_path(self, file_path):
        with open(file_path, 'rb') as file:
            data = file.read()
            encrypted_data = self.cipher.encrypt(data)

        encrypted_file_path = file_path + ".enc"
        with open(encrypted_file_path, 'wb') as encrypted_file:
            encrypted_file.write(encrypted_data)

    def decrypt_file_path(self, file_path):
        with open(file_path, 'rb') as encrypted_file:
            encrypted_data = encrypted_file.read()
            decrypted_data = self.cipher.decrypt(encrypted_data)

        decrypted_file_path = file_path[:-4]
        with open(decrypted_file_path, 'wb') as decrypted_file:
            decrypted_file.write(decrypted_data)

if __name__ == "__main__":
    root = tk.Tk()
    app = FileEncryptionApp(root)
    root.mainloop()


In [20]:
#importing packages
import tkinter as tk
from tkinter import filedialog, messagebox
from cryptography.fernet import Fernet
from PIL import Image, ImageTk
import io
import os

class FileEncryptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Application for Encryption and Decryption")

        # Set a custom background color split half-and-half between white and maroon
        self.root.configure(bg="#FFFFFF")
        self.canvas = tk.Canvas(root, bg="#800000", height=self.root.winfo_screenheight(), width=self.root.winfo_screenwidth() / 2)
        self.canvas.pack(side=tk.LEFT)

        # Generate a random key for encryption/decryption
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#000080", 'fg': "white", 'font': ('Helvetica', 12)}

        # Create buttons for file operations
        self.encrypt_button = tk.Button(root, text="Encrypt a Text File", command=self.encrypt_file, **button_style)
        self.encrypt_button.pack(pady=10)

        self.decrypt_button = tk.Button(root, text="Decrypt a Text File", command=self.decrypt_file, **button_style)
        self.decrypt_button.pack(pady=10)

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.pack(pady=10)

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.pack(pady=10)

        # Create buttons for image operations
        self.encrypt_image_button = tk.Button(root, text="Encrypt a JPG Image", command=self.encrypt_image, **button_style)
        self.encrypt_image_button.pack(pady=10)

        self.decrypt_image_button = tk.Button(root, text="Decrypt a JPG Image", command=self.decrypt_image, **button_style)
        self.decrypt_image_button.pack(pady=10)

        # Create a label for displaying encrypted/decrypted images
        self.image_label = tk.Label(root, text="Encrypted/Decrypted Image:", font=('Helvetica', 14), bg="#FFFFFF")
        self.image_label.pack()

    def encrypt_file(self):
        file_path = filedialog.askopenfilename(title="Select File to Encrypt", filetypes=[("All Files", "*.*")])
        if file_path:
            with open(file_path, 'rb') as file:
                data = file.read()
                encrypted_data = self.cipher.encrypt(data)

            encrypted_file_path = filedialog.asksaveasfilename(title="Save Encrypted File", defaultextension="enc",
                                                                filetypes=[("Encrypted Files", "*.enc")])
            if encrypted_file_path:
                with open(encrypted_file_path, 'wb') as encrypted_file:
                    encrypted_file.write(encrypted_data)
                    messagebox.showinfo("Success", "File encrypted successfully.")

    def decrypt_file(self):
        file_path = filedialog.askopenfilename(title="Select File to Decrypt", filetypes=[("Encrypted Files", "*.enc")])
        if file_path:
            with open(file_path, 'rb') as encrypted_file:
                encrypted_data = encrypted_file.read()
                decrypted_data = self.cipher.decrypt(encrypted_data)

            decrypted_file_path = filedialog.asksaveasfilename(title="Save Decrypted File",
                                                                filetypes=[("All Files", "*.*")])
            if decrypted_file_path:
                with open(decrypted_file_path, 'wb') as decrypted_file:
                    decrypted_file.write(decrypted_data)
                    messagebox.showinfo("Success", "File decrypted successfully.")

    def encrypt_image(self):
        image_path = filedialog.askopenfilename(title="Select Image to Encrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            original_image = Image.open(image_path)
            original_image.thumbnail((300, 300))
            original_image_tk = ImageTk.PhotoImage(original_image)
            self.image_label.config(image=original_image_tk)
            self.image_label.image = original_image_tk

            with open(image_path, 'rb') as image_file:
                image_data = image_file.read()
                encrypted_data = self.cipher.encrypt(image_data)

            encrypted_image = Image.open(io.BytesIO(encrypted_data))
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            messagebox.showinfo("Success", "Image encrypted successfully.")

    def decrypt_image(self):
        image_path = filedialog.askopenfilename(title="Select Image to Decrypt", filetypes=[("Image Files", "*.jpg;*.jpeg")])
        if image_path:
            encrypted_image = Image.open(image_path)
            encrypted_image.thumbnail((300, 300))
            encrypted_image_tk = ImageTk.PhotoImage(encrypted_image)
            self.image_label.config(image=encrypted_image_tk)
            self.image_label.image = encrypted_image_tk

            with open(image_path, 'rb') as encrypted_image_file:
                encrypted_data = encrypted_image_file.read()
                decrypted_data = self.cipher.decrypt(encrypted_data)

            decrypted_image = Image.open(io.BytesIO(decrypted_data))
            decrypted_image.thumbnail((300, 300))
            decrypted_image_tk = ImageTk.PhotoImage(decrypted_image)
            self.image_label.config(image=decrypted_image_tk)
            self.image_label.image = decrypted_image_tk

            messagebox.showinfo("Success", "Image decrypted successfully.")

    def encrypt_folder(self):
        folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
        if folder_path:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.encrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
        if folder_path:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    self.decrypt_file_path(file_path)

            messagebox.showinfo("Success", "Folder decrypted successfully.")

    def encrypt_file_path(self, file_path):
        with open(file_path, 'rb') as file:
            data = file.read()
            encrypted_data = self.cipher.encrypt(data)

        encrypted_file_path = file_path + ".enc"
        with open(encrypted_file_path, 'wb') as encrypted_file:
            encrypted_file.write(encrypted_data)

    def decrypt_file_path(self, file_path):
        with open(file_path, 'rb') as encrypted_file:
            encrypted_data = encrypted_file.read()
            decrypted_data = self.cipher.decrypt(encrypted_data)

        decrypted_file_path = file_path[:-4]
        with open(decrypted_file_path, 'wb') as decrypted_file:
            decrypted_file.write(decrypted_data)

if __name__ == "__main__":
    root = tk.Tk()
    app = FileEncryptionApp(root)
    root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/2153574520.py", line 92, in encrypt_image
    encrypted_image = Image.open(io.BytesIO(encrypted_data))
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 3030, in open
    raise UnidentifiedImageError(
PIL.UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002AEAC516720>
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/2153574520.py", line 92, in encrypt_image
    encrypted_image = Image.open(io.BytesIO(encrypted_data))
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 3030, in open
    raise Unidentifi

In [21]:
#importing packages
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os

class FolderEncryptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Folder Encryption and Decryption")

        # Create and configure the style for buttons
        button_style = {'padx': 10, 'pady': 5, 'bg': "#000080", 'fg': "white", 'font': ('Helvetica', 12)}

        # Create buttons for folder operations
        self.encrypt_folder_button = tk.Button(root, text="Encrypt a Folder", command=self.encrypt_folder, **button_style)
        self.encrypt_folder_button.pack(pady=10)  # Use pack for vertical alignment

        self.decrypt_folder_button = tk.Button(root, text="Decrypt a Folder", command=self.decrypt_folder, **button_style)
        self.decrypt_folder_button.pack(pady=10)  # Use pack for vertical alignment

        # Split the background between white and maroon
        self.root.configure(bg="#FFFFFF")
        self.canvas = tk.Canvas(root, bg="#800000", height=self.root.winfo_screenheight() / 2, width=self.root.winfo_screenwidth())
        self.canvas.pack()

    def get_password(self):
        # Prompt user for password
        password = simpledialog.askstring("Password", "Enter password:", show='*')
        return password.encode('utf-8')

    def derive_key(self, password):
        # Derive a key from the password using PBKDF2HMAC
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'\xae\xd6c}\x93\xa6\xcc\xc2\n\xd4\x98\x16\xc6y\x8f',  # Use a different salt for each application
            iterations=100000,
            backend=default_backend()
        )
        key = Fernet(kdf.derive(password))
        return key

    def encrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Encrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and encrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as file:
                            data = file.read()
                            encrypted_data = cipher.encrypt(data)

                        encrypted_file_path = file_path + ".enc"
                        with open(encrypted_file_path, 'wb') as encrypted_file:
                            encrypted_file.write(encrypted_data)

                messagebox.showinfo("Success", "Folder encrypted successfully.")

    def decrypt_folder(self):
        password = self.get_password()
        if password:
            folder_path = filedialog.askdirectory(title="Select Folder to Decrypt")
            if folder_path:
                key = self.derive_key(password)
                cipher = Fernet(key)

                # Walk through the selected folder and decrypt each file
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as encrypted_file:
                            encrypted_data = encrypted_file.read()
                            decrypted_data = cipher.decrypt(encrypted_data)

                        decrypted_file_path = file_path[:-4]
                        with open(decrypted_file_path, 'wb') as decrypted_file:
                            decrypted_file.write(decrypted_data)

                messagebox.showinfo("Success", "Folder decrypted successfully.")

if __name__ == "__main__":
    root = tk.Tk()
    app = FolderEncryptionApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/4019075837.py", line 52, in encrypt_folder
    key = self.derive_key(password)
  File "C:\Users\dabsyed\AppData\Local\Temp/ipykernel_182064/4019075837.py", line 44, in derive_key
    key = Fernet(kdf.derive(password))
  File "C:\ProgramData\Anaconda3\lib\site-packages\cryptography\fernet.py", line 32, in __init__
    key = base64.urlsafe_b64decode(key)
  File "C:\ProgramData\Anaconda3\lib\base64.py", line 133, in urlsafe_b64decode
    return b64decode(s)
  File "C:\ProgramData\Anaconda3\lib\base64.py", line 87, in b64decode
    return binascii.a2b_base64(s)
binascii.Error: Invalid base64-encoded string: number of data characters (5) cannot be 1 more than a multiple of 4
